In [1]:
import os
import openai
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()
openai.api_type = os.environ["API_TYPE"]
openai.api_base = os.environ["API_BASE"]
openai.api_version = os.environ["API_VERSION"]
openai.api_key = os.environ["API_KEY"]

engine = os.environ["API_ENGINE"]

In [2]:
import json

# 検索結果を返す関数の作成
def get_search_result(keyword, when, unit="d"):
    result = {
        "result": [
            {"title": "1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました"},
            {"title": "1929年（昭和4年）八重洲口が開設されました"},
            {"title": "2014年（平成26年）東京駅開業100周年を迎えました"},
        ]
    }

    return json.dumps(result) 

# 関数情報の作成
functions=[
    {
        # 関数名を指定
        "name": "get_search_result",
        # 関数の概要。これをもとに言語モデルが関数使用を判断するためなるべく詳細に記述する。
        "description": "指定したキーワードの検索結果を取得する",
        "parameters": {
            # 関数の型を指定(string, number, arrayなど)
            "type": "object",
            # 関数のパラメーターを記述
            "properties": {
                # 引数名
                "keyword": {
                    # 引数の型
                    "type": "string",
                    # 引数の意味。これをもとに言語モデルが関数使用を判断するためなるべく詳細に記述する。
                    "description": "キーワード",
                },
                "when": {
                    "type": "number",
                    "description": "日付や時間の範囲"
                },
                "unit": {
                    "type": "string",
                    "enum": ["d", "h", "y"]
                },
            },
            # 必須パラメーターをリストで指定
            "required": ["keyword"],
        },
    }
]

In [3]:
# Chat API呼び出し①
question="東京駅の駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

response=openai.ChatCompletion.create(
    engine=engine,
    messages=[
        {"role": "user", "content": question},
    ],
    functions=functions,
    function_call="auto",
)

#response
response["choices"][0]["message"]["function_call"]["arguments"]

'{\n  "keyword": "東京駅のイベント",\n  "when": 1,\n  "unit": "m"\n}'

In [4]:
# Python 関数呼び出しとChatAPI呼び出し②

# 言語モデルからの回答メッセージを取得
message: dict[str, str] = response["choices"][0]["message"]
print(f"メッセージ: {message}")

# モデルが関数呼出と判断した
if message.get("function_call"):
    # 関数名の取得
    function_name = message["function_call"]["name"]
    print(f"関数名：{function_name}")
    # 引数の取得
    arguments = json.loads(message["function_call"]["arguments"])
    print(f"引数：{arguments}")

    # 関数名で実行する関数を判断
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )
    print(f"関数の実行結果：{function_response}")

    # 関数の実行結果をmessagesに加えて言語モデルを呼出
    response2 = openai.ChatCompletion.create(
        engine=engine,
        messages=[
            {"role": "user", "content": question},
            #message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print(response2.choices[0]["message"]["content"].strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print(response.choices[0]["message"]["content"].strip())

メッセージ: {
  "role": "assistant",
  "function_call": {
    "name": "get_search_result",
    "arguments": "{\n  \"keyword\": \"\u6771\u4eac\u99c5\u306e\u30a4\u30d9\u30f3\u30c8\",\n  \"when\": 1,\n  \"unit\": \"m\"\n}"
  }
}
関数名：get_search_result
引数：{'keyword': '東京駅のイベント', 'when': 1, 'unit': 'm'}
関数の実行結果：{"result": [{"title": "1914\u5e74\uff08\u5927\u6b633\u5e74\uff09\u4e2d\u592e\u505c\u8eca\u5834\u6539\u3081\u300c\u6771\u4eac\u99c5\u300d\u3068\u3057\u3066\u55b6\u696d\u3092\u958b\u59cb\u3057\u307e\u3057\u305f"}, {"title": "1929\u5e74\uff08\u662d\u548c4\u5e74\uff09\u516b\u91cd\u6d32\u53e3\u304c\u958b\u8a2d\u3055\u308c\u307e\u3057\u305f"}, {"title": "2014\u5e74\uff08\u5e73\u621026\u5e74\uff09\u6771\u4eac\u99c5\u958b\u696d100\u5468\u5e74\u3092\u8fce\u3048\u307e\u3057\u305f"}]}
最近1ヶ月以内の東京駅の駅のイベントについての検索結果は以下の通りです：

1. 1914年（大正3年）中央停車場改め「東京駅」の開業を記念して、2021年11月1日から営業を開始しました。
   （タイトル: 1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました）

2. 1929年（昭和4年）八重洲口が開設されました。
   （タイトル: 1929年（昭和4年）八重洲口が開設されました）

3. 2014年（平成